# Installation
```bash
# create an environment
conda create --name lec4 python=3.9
conda activate lec4
# install pytorch. This one can use GPU acceleration on mac
conda install pytorch -c pytorch-nightly 
# install jupyter
conda install -n lec4 ipykernel --update-deps --force-reinstall
conda install -c anaconda jupyter
# installing huggingface libraries
conda install transformers
conda install datasets
```

# Walkthrough

In [1]:
import numpy as np
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import TrainingArguments, Trainer

In [2]:
# piqa is a physical reasoning question answer dataset that provides two potential (and usually similar)
# responses to each question as well as a label for the right answer
dataset = load_dataset("piqa")
train = dataset['train']
# using validation because test appears to not be labelled
test = dataset['validation']

Reusing dataset piqa (C:\Users\cjj90\.cache\huggingface\datasets\piqa\plain_text\1.1.0\6c611c1a9bf220943c4174e117d3b660859665baf1d43156230116185312d011)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# only hold onto the right solution
def consolidate(row):
    return {'sol': (row['sol1'] if row['label'] == 0 else row['sol2'])}
train = train.map(consolidate, remove_columns = ['label', 'sol1', 'sol2'])
test = test.map(consolidate, remove_columns = ['label', 'sol1', 'sol2'])

Parameter 'function'=<function consolidate at 0x0000025F890F4160> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/16113 [00:00<?, ?ex/s]

  0%|          | 0/1838 [00:00<?, ?ex/s]

In [4]:
train[0]

{'goal': "When boiling butter, when it's ready, you can",
 'sol': 'Pour it into a jar'}

In [5]:
# load config without weights
model_name = "bert-base-uncased" 
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
config = AutoConfig.from_pretrained(model_name)
model = AutoModelForCausalLM.from_config(config)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [6]:
def tokenize_function(examples):
    return tokenizer(examples['goal'], examples['sol'], truncation=True)

tokenized_train = train.map(tokenize_function, batched = True, remove_columns = ['goal', 'sol'])
tokenized_test = test.map(tokenize_function, batched = True, remove_columns = ['goal', 'sol'])

  0%|          | 0/17 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [7]:
block_size = 128

def group_texts(examples):
    # repeat concatenation for input_ids and other keys
    concatenated_examples = {k: sum(examples[k], []) for k in
                            examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size

    # populate each of input_ids and other keys 
    result = {
        k: [t[i : i + block_size] for i in range(0,
            total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    # add labels because we'll need it as the output
    result["labels"] = result["input_ids"].copy()
    return result

In [8]:
lm_train = tokenized_train.map(group_texts, batched = True, batch_size = 1000)
lm_test = tokenized_test.map(group_texts, batched = True, batch_size = 1000)

  0%|          | 0/17 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [9]:
tokenizer.decode(lm_train[0]['input_ids'])

"[CLS] when boiling butter, when it's ready, you can [SEP] pour it into a jar [SEP] [CLS] to permanently attach metal legs to a chair, you can [SEP] weld the metal together to get it to stay firmly in place [SEP] [CLS] how do you indent something? [SEP] leave a space before starting the writing [SEP] [CLS] how do you shake something? [SEP] move it up and down and side to side quickly. [SEP] [CLS] clean tires [SEP] pour water, scrape off caked on dirt. use a steel wool to clean out crevices and narrow spaces. [SEP] [CLS] how do you taste something? [SEP] place it in your mouth to taste"

In [10]:
small_train_dataset = lm_train.shuffle(seed=42).select(range(100))
small_eval_dataset = lm_test.shuffle(seed=42).select(range(100))

In [11]:
training_args = TrainingArguments(
    f"{model_name}-piqa",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    weight_decay = 0.01,
    push_to_hub = False,
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = small_train_dataset,
    eval_dataset = small_eval_dataset,
)

trainer.train()

c:\Users\cjj90\anaconda3\envs\pset2\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 100
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 39


  0%|          | 0/39 [00:00<?, ?it/s]

***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 8.992371559143066, 'eval_runtime': 21.4162, 'eval_samples_per_second': 4.669, 'eval_steps_per_second': 0.607, 'epoch': 1.0}


...

***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


  0%|          | 0/13 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 8.52519702911377, 'eval_runtime': 18.2565, 'eval_samples_per_second': 5.478, 'eval_steps_per_second': 0.712, 'epoch': 3.0}
{'train_runtime': 247.6237, 'train_samples_per_second': 1.212, 'train_steps_per_second': 0.157, 'train_loss': 8.976375482021234, 'epoch': 3.0}


TrainOutput(global_step=39, training_loss=8.976375482021234, metrics={'train_runtime': 247.6237, 'train_samples_per_second': 1.212, 'train_steps_per_second': 0.157, 'train_loss': 8.976375482021234, 'epoch': 3.0})

In [12]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


  0%|          | 0/13 [00:00<?, ?it/s]

Perplexity: 5040.18


In [13]:
tokenizer.save_pretrained(f"{model_name}-piqa")

tokenizer config file saved in bert-base-uncased-piqa\tokenizer_config.json
Special tokens file saved in bert-base-uncased-piqa\special_tokens_map.json


('bert-base-uncased-piqa\\tokenizer_config.json',
 'bert-base-uncased-piqa\\special_tokens_map.json',
 'bert-base-uncased-piqa\\vocab.txt',
 'bert-base-uncased-piqa\\added_tokens.json',
 'bert-base-uncased-piqa\\tokenizer.json')

In [14]:
start_text = ("A speedrun is a playthrough of a video game, \
or section of a video game, with the goal of \
completing it as fast as possible. Speedruns \
often follow planned routes, which may incorporate sequence \
breaking, and might exploit glitches that allow sections to \
be skipped or completed more quickly than intended. ")

prompt = "What is the"
inputs = tokenizer(
     start_text + prompt,
     add_special_tokens=False,
     return_tensors="pt")["input_ids"]

prompt_length = len(tokenizer.decode(inputs[0]))
outputs = model.generate(
     inputs,
     max_length=100,
     do_sample=True,
     top_k=50,
     top_p=0.95,
     temperature=0.9,
     num_return_sequences=3)

generated = prompt + tokenizer.decode(outputs[0])[prompt_length + 1:]

print(tokenizer.decode(outputs[0]))

a speedrun is a playthrough of a video game, or section of a video game, with the goal of completing it as fast as possible. speedruns often follow planned routes, which may incorporate sequence breaking, and might exploit glitches that allow sections to be skipped or completed more quickly than intended. what is the [SEP] how [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] josiah [SEP] [SEP] [CLS] [SEP] [CLS] [SEP] [SEP] [SEP] [CLS] [SEP] can [SEP] [SEP] to [SEP] [SEP]⁰ [SEP] [CLS] [SEP] [SEP] almond [SEP]
